In [1]:
# Import Librariesfrom keras.models import Sequential
from keras.layers import Dense
import re
import warnings
from keras.models import load_model
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import time
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.preprocessing import label_binarize
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
import seaborn as sns
from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.metrics import roc_curve,auc
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from keras import initializers
import keras
import numpy as np
import pandas as pd
import time
import cv2
import matplotlib.pyplot as plt
import os
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform
from keras.models import load_model

Using TensorFlow backend.


In [68]:
data = pd.read_csv('amazon_reviews_cleaned.csv')

In [69]:
data

,Unnamed: 0,ProfileName,HelpfulnessNumerator,Score,Review
0,0,Jesse Mathers,1,0,really want like cant
1,1,"L. Jonte ""Artist, Writer, Flibbertigibbet, Ed...",0,0,thanks .
2,2,H. Yu,1,0,literally cheap tea
3,3,Mark Twain,1,0,shame obscure weight highlight bargain price .
4,4,"Elrond ""Elrond""",2,0,mystify positive review
5,5,Chip Green,0,0,hernia induce cat litter
6,6,Fred K. White,0,0,white stash tea
7,7,"Van Zandt ""Galloping Gourmet too""",1,0,nothing original
8,8,jcosroque1971,0,0,kid
9,9,Phillip Banks,3,0,terrible walden farm ranch dress


In [70]:
# reviews_text = data["Text"].values
# reviews_title = data["Summary"].values
reviews = data["Review"].values
reviews_rating = data["Score"].values

In [71]:
reviews_rating

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [72]:
# nltk libraries

stop_words = set(stopwords.words('english'))
lemmatiser = WordNetLemmatizer()

In [73]:
import nltk
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
stop_words=set(stopwords.words('english'))
lemmatiser=WordNetLemmatizer()

In [74]:
# returns the Part-Of-Speech(pos) tag which is required as an argument for
# lemmatizer for determiing the context of the word for lemmatization
def get_wordnet_pos(pos_tag):

    if pos_tag.startswith('J'):  
        return 'a' # adjective
    elif pos_tag.startswith('V'): 
        return 'v' # verb
    elif pos_tag.startswith('R'):
        return 'r' # adverb
    else:           
        return 'n' # fallback to noun

In [75]:
#Preprocess the entire text
def preprocess_text(text):
    
    # lowercase the entire text
    text = text.lower()
    
    # remove all html tags from dataset
    text = re.sub(r'<.*?>',"",text)
    
    # Replace special characters and punctuation by a 
    # blank ( except for full stop(.))
    text = re.sub('[^a-z0-9 \.]', '', text)
    
    # split the text into words
    words = word_tokenize(text) 
    
    # remove stop words like to, an, the, of, over, his, her, there, their ...
    words = [w for w in words if w =="not" or not w in stop_words ]
    
    # get pos tagging for words, required by lemmatizer
    words_pos = pos_tag(words) 
    
    lemmatized_words = []
    
    for word, tag in words_pos:
        pos = get_wordnet_pos(tag)
        lemmatized_word = lemmatiser.lemmatize(word, pos=pos)
        lemmatized_words.append(lemmatized_word)
    
    text = " ".join(lemmatized_words)
    
    return text

In [79]:
DATA_SIZE = reviews.size
labels = np.array(data['Score'])

In [82]:
reviews

array(['really want like cant', 'thanks .', 'literally cheap tea', ...,
       'great stuff', 'fanrastic', 'best best wheatfree pasta'],
      dtype=object)

In [94]:
for s in reviews:
    print(len(s.split()))

4
2
3
7
3
4
3
2
1
5
1
2
2
1
3
2
4
1
2
5
2
4
1
6
3
2
3
3
2
2
1
3
4
1
7
6
1
2
5
4
3
3
2
3
2
3
2
1
6
2
5
2
2
3
2
3
7
3
1
2
4
2
1
4
3
1
2
3
3
2
6
2
6
4
5
1
1
3
3
3
5
2
6
2
3
3
3
4
2
3
6
3
7
7
9
2
4
5
1
4
1
2
3
1
7
4
1
3
4
4
6
3
6
3
4
3
4
1
4
7
2
5
1
2
2
2
3
3
10
3
5
2
6
4
3
1
4
2
2
3
3
1
3
2
1
2
2
3
2
5
1
2
3
3
3
3
4
1
2
2
2
2
4
2
2
4
2
3
4
2
1
6
3
4
4
1
3
2
2
4
3
1
2
2
2
2
4
4
2
3
2
3
1
8
3
2
3
4
5
2
1
1
3
4
7
3
3
2
6
5
2
2
4
1
2
1
3
2
6
3
1
1
3
4
3
3
6
3
2
1
2
1
5
6
2
2
3
3
4
2
1
8
2
5
2
5
2
2
3
4
5
7
4
4
2
2
2
3
1
2
5
2
2
1
2
3
2
4
2
4
3
4
3
3
3
4
3
2
5
1
3
3
5
4
3
3
4
2
2
7
3
1
1
7
5
1
1
1
4
4
2
2
1
9
3
3
2
3
2
2
4
3
4
1
4
1
2
3
2
7
5
4
2
6
8
3
3
2
1
6
4
3
7
1
3
2
2
4
3
2
2
6
6
2
2
4
6
4
4
4
3
3
1
3
4
3
4
5
5
2
2
4
4
3
3
2
8
4
8
1
6
2
6
3
1
6
5
3
2
3
3
4
9
7
7
7
2
2
1
1
4
4
4
3
1
3
4
1
3
13
2
5
6
1
4
6
2
4
5
3
3
6
3
3
2
6
2
3
1
2
3
2
5
2
2
2
3
6
5
3
2
2
3
3
5
3
2
3
4
3
2
2
8
5
4
2
2
7
4
1
4
4
2
2
4
1
4
4
7
2
4
2
3
5
1
3
5
4
3
1
4
2
4
2
1
1
2
4
6
4
7
8
4
1
1
3
7
2
3
3
2
7
2
2
3
3
2
3
3


AttributeError: 'float' object has no attribute 'split'

In [101]:
reviews

array(['really want like cant', 'thanks .', 'literally cheap tea', ...,
       'great stuff', 'fanrastic', 'best best wheatfree pasta'],
      dtype=object)